In [ ]:
# imports

# imports

In [ ]:
!pip install -U ipykernel
!pip install missingno
!pip install sweetviz
!pip install dtale
!pip install dash
!pip install vizro

  Using cached jedi-0.19.1-py2.py3-none-any.whl.metadata (22 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 117.2/117.2 kB 1.6 MB/s eta 0:00:00
Using cached jedi-0.19.1-py2.py3-none-any.whl (1.6 MB)
  Attempting uninstall: ipykernel
    Found existing installation: ipykernel 5.5.6
    Uninstalling ipykernel-5.5.6:
      Successfully uninstalled ipykernel-5.5.6
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-colab 1.0.0 requires ipykernel==5.5.6, but you have ipykernel 6.29.5 which is incompatible.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 15.1/15.1 MB 58.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.3/62.3 kB 1.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 642.7/642.7 kB 10.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [ ]:
import missingno as msno
import dtale
import sweetviz as sv


from vizro import Vizro
import vizro.plotly.express as px
from vizro.tables import dash_ag_grid
import vizro.models as vm
from pathlib import Path
from vizro.actions import filter_interaction
from vizro.models.types import capture

import numpy as np

#padaria_modelo = 453832840298988785


In [ ]:
import dtale.app as dtale_app
dtale_app.USE_COLAB = True
dtale_app.USE_NGROK = True

## Load your data

The code below will download all the provided data. Sit back and relax while Google Colab takes care of it.

In [ ]:
%%capture

# install and load necessary Libraries
!pip install pyarrow
!pip install gdown

import gdown
import pandas as pd

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

MessageError: Error: credential propagation was unsuccessful

In [ ]:
data = {
    'bank': "1dzL_SWBkBs5xrUxuGQTm04oe3USgkL9u",    # banking data
    'sales': "1QK-VgSU3AxXUw330KjYFUj8S9hzKJsG6",   # sales data
    'mcc': "1JN0bR84sgZ_o4wjKPBUmz45NeEEkVgt7",     # mcc description
}

In [ ]:
# Download all files from Google Drive
for name, file_id in data.items():
    gdown.download(f'https://drive.google.com/uc?id={file_id}', name + '.parquet', quiet=False)

In [ ]:
# Read all files and store on a dictionary of pandas dataframes
df = {}
for name in data.keys():
    df[name] = pd.read_parquet(name + '.parquet')

## Data Cleaning

You can access each dataframe inside the `df` dictionary using the code shown below

In [ ]:
mcc   = df["mcc"]
bank  = df['bank']
sales = df['sales']

lista_ids_bank            = df['bank'].document_id.unique()
lista_ids_sales           = df['sales'].document_id.unique()
lista_ids_apenas_em_sales = sales[~sales["document_id"].isin(lista_ids_bank)]["document_id"].unique()

mcc_simple = mcc[["mcc","edited_description"]]
mcc_simple.columns = ["mcc","sector"]
sales = sales.merge(mcc_simple,on="mcc",how="left")

dict_id_branch = sales.drop_duplicates("document_id")[['document_id','sector']]


bank.loc[bank['type'] ==  'pix_out', 'value'] *= -1
bank =  bank.merge(dict_id_branch,on='document_id',how='left')
bank = bank.sort_values(['document_id',"date_time"])
bank["document_id"] = bank["document_id"].astype(str)
bank["counterparty_document"] = bank["counterparty_document"].astype(str)


sample_store = str(453832840298988785)
sales['document_id'] = sales['document_id'].astype(str)



In [ ]:
bank_day = bank.set_index("date_time").groupby(['sector','document_id',pd.Grouper(freq='d'), 'type']).agg("sum").reset_index().iloc[:,:-1]
bank_week = bank.set_index("date_time").groupby(['sector','document_id',pd.Grouper(freq='w'), 'type']).agg("sum").reset_index().iloc[:,:-1]

In [ ]:
# daily

outflow_day = bank_day[bank_day["type"] == "pix_out"]
inflow_day = bank_day[bank_day["type"] == "pix_in"]
net_day = bank_day.copy()


outflow_day['cum_out'] = outflow_day.groupby("document_id")['value'].cumsum()
inflow_day['cum_in'] = inflow_day.groupby("document_id")['value'].cumsum()
net_day['cum_net'] = net_day.groupby("document_id")['value'].cumsum()

outflow_day.columns = ['sector', 'document_id', 'date_time', 'type', 'out', 'cum_out']
inflow_day.columns = ['sector', 'document_id', 'date_time', 'type', 'in', 'cum_in']
net_day.columns = ['sector', 'document_id', 'date_time', 'type', 'net', 'cum_net']

# weekly

outflow_week = bank_week[bank_week["type"] == "pix_out"]
inflow_week = bank_week[bank_week["type"] == "pix_in"]
net_week = bank_week.copy()

outflow_week['cum_out'] = outflow_week.groupby("document_id")['value'].cumsum()
inflow_week['cum_in'] = inflow_week.groupby("document_id")['value'].cumsum()
net_week['cum_net'] = net_week.groupby("document_id")['value'].cumsum()

outflow_week.columns = ['sector', 'document_id', 'date_time', 'type', 'out', 'cum_out']
inflow_week.columns = ['sector', 'document_id', 'date_time', 'type', 'in', 'cum_in']
net_week.columns = ['sector', 'document_id', 'date_time', 'type', 'net', 'cum_net']

In [ ]:
day = net_day.merge(outflow_day[['document_id','date_time','out','cum_out']],on=['document_id','date_time'],how='left').merge(inflow_day[['document_id','date_time','in','cum_in']],on=['document_id','date_time'],how='left')
day['cum_in'] = day.groupby('document_id')['cum_in'].ffill()
day['cum_out'] = day.groupby('document_id')['cum_out'].ffill()

week = net_week.merge(outflow_week[['document_id','date_time','out','cum_out']],on=['document_id','date_time'],how='left').merge(inflow_week[['document_id','date_time','in','cum_in']],on=['document_id','date_time'],how='left')
week['cum_in']  = week.groupby('document_id')['cum_in'].ffill()
week['cum_out'] = week.groupby('document_id')['cum_out'].ffill()

## Cash Flow

#### Dash Cash Flow

In [ ]:
cum_day  = day[['sector','document_id','date_time','cum_net','cum_out','cum_in']].melt(id_vars= ['sector','document_id','date_time'])
cum_week = week[['sector','document_id','date_time','cum_net','cum_out','cum_in']].melt(id_vars= ['sector','document_id','date_time'])

cum_day['id_var']  = cum_day['document_id'] + '_' + cum_day['variable']
cum_week['id_var'] = cum_week['document_id'] + '_'+ cum_week['variable']

px.line(cum_day, x="date_time", y="value",color="id_var",custom_data=["sector","document_id"])

In [ ]:
day.to_excel('day.xlsx')
week.to_excel('week.xlsx')

outflow_week.to_excel('outflow_week.xlsx')
inflow_week.to_excel('inflow_week.xlsx')
net_week.to_excel('net_week.xlsx')

outflow_day.to_excel('outflow_day.xlsx')
inflow_day.to_excel('inflow_day.xlsx')
net_day.to_excel('net_day.xlsx')

In [440]:
cum_day.to_excel('cum_day.xlsx')
cum_week.to_excel('cum_week.xlsx')

In [ ]:
# in
px.line(inflow_day, x="date_time", y="in",color='document_id',custom_data=["sector","document_id"])

# out
px.line(outflow_day, x="date_time", y="out",color='document_id',custom_data=["sector","document_id"])

In [442]:
Vizro._reset()
page = vm.Page(
    title="Strats",
    layout=vm.Layout(grid=[[0, 0],[1,2]]),
    components=[
        vm.Graph(id="cumsum", figure=px.line(cum_day, x="date_time", y="value",color="id_var",custom_data=["sector","document_id"])),
        vm.Graph(id="in", figure=px.line(inflow_day, x="date_time", y="in",color='document_id',custom_data=["sector","document_id"])),
        vm.Graph(id="out", figure=px.line(outflow_day, x="date_time", y="out",color='document_id',custom_data=["sector","document_id"])),
],
    controls=[
        vm.Filter(column="sector", selector=vm.Dropdown(value=["ALL"])),
        vm.Filter(column="document_id", selector=vm.Dropdown(value=["ALL"])),
        vm.Filter(column="date_time"),
    ],
)

dashboard = vm.Dashboard(pages=[page],theme="vizro_light")

Vizro().build(dashboard).run()

/usr/local/lib/python3.10/dist-packages/_plotly_utils/basevalidators.py:105: FutureWarning:

The behavior of DatetimeProperties.to_pydatetime is deprecated, in a future version this will return a Series containing python datetime objects instead of an ndarray. To retain the old behavior, call `np.array` on the result

/usr/local/lib/python3.10/dist-packages/_plotly_utils/basevalidators.py:105: FutureWarning:

The behavior of DatetimeProperties.to_pydatetime is deprecated, in a future version this will return a Series containing python datetime objects instead of an ndarray. To retain the old behavior, call `np.array` on the result

/usr/local/lib/python3.10/dist-packages/_plotly_utils/basevalidators.py:105: FutureWarning:

The behavior of DatetimeProperties.to_pydatetime is deprecated, in a future version this will return a Series containing python datetime objects instead of an ndarray. To retain the old behavior, call `np.array` on the result

/usr/local/lib/python3.10/dist-packages/

LookupError: <ContextVar name='callback_context' at 0x7b8f701ee390>

In [ ]:
%pip install -U ipykernel

### Forecast

### Seasonality)

### Forecasting

## Sales

### Growth

#### Customer Segmentation (FRM)

In [ ]:
client_profiles = []
new_sales = []
df_breakdown = []


recency_scores = [5, 4, 3, 2, 1]
frequency_scores = [1, 2, 3, 4, 5]
monetary_scores = [1, 2, 3, 4, 5]
segment_labels = ['Low-Value', 'Mid-Value', 'High-Value']

for id in sales.document_id.unique():

  sales_pad = sales[sales['document_id'] == id].copy()
  sales_pad = sales_pad[sales_pad['card_number'] != ""]

  setor = sales_pad['sector'].values[0]
  fim    = sales_pad['date_time'].max().to_pydatetime()
  inicio = sales_pad['date_time'].min().to_pydatetime()
  media = sales_pad['value'].mean()
  n_clientes = len(sales_pad['card_number'].unique())

  sales_pad['Recency'] = (fim - sales_pad['date_time']).dt.days

  frequency_data = sales_pad.groupby('card_number')['value'].count().reset_index()
  frequency_data.columns = ['card_number','Frequency']
  sales_pad = sales_pad.merge(frequency_data,on='card_number',how='left')

  monetary_data = sales_pad.groupby('card_number')['value'].sum().reset_index()
  monetary_data.columns = ['card_number','MonetaryValue']
  sales_pad = sales_pad.merge(monetary_data,on='card_number',how='left')

  sales_pad['RecencyScore'] = pd.cut(sales_pad['Recency'], bins = 5, labels= recency_scores)
  sales_pad['FrequencyScore'] = pd.cut(sales_pad['Frequency'], bins = 5, labels= frequency_scores)
  sales_pad['MonetaryScore'] = pd.cut(sales_pad['MonetaryValue'], bins= 5, labels= monetary_scores)

  sales_pad['RecencyScore']   = sales_pad['RecencyScore'].astype(int)
  sales_pad['FrequencyScore'] = sales_pad['FrequencyScore'].astype(int)
  sales_pad['MonetaryScore']  = sales_pad['MonetaryScore'].astype(int)

  sales_pad['RFM_Score'] = sales_pad['RecencyScore'] + sales_pad['FrequencyScore'] + sales_pad['MonetaryScore']

  #sales_pad['RFM Segment'] = pd.qcut(data['RFM_Score'], q=3, labels=segment_labels)

  sales_pad['Value Segment'] = pd.qcut(sales_pad['RFM_Score'].rank(method='first'), 3,labels=segment_labels)

  #sales_pad.loc[sales_pad['RFM Segment'] >= 9, 'RFM Segment'] = 'Champions'
  #sales_pad.loc[(sales_pad['RFM Segment'] >= 7) & (sales_pad['RFM Segment'] < 11), 'RFM Customer Segments'] = 'Potential Loyalists'
  #sales_pad.loc[(sales_pad['RFM Segment'] < 8) & (sales_pad['RFM Segment'] ), 'RFM Customer Segments'] = 'At-Risk Customers'

  segment_counts = sales_pad.drop_duplicates("card_number")['Value Segment'].value_counts().reset_index()
  segment_counts.columns = ['Value Segment', 'Count']

  segment_counts["%"] = segment_counts["Count"] / n_clientes
  segment_counts["sector"] = setor
  segment_counts["document_id"] = str(id)

  sales_pad['RFM Customer Segments'] = ''

  sales_pad.loc[sales_pad['RFM_Score'] >= 9, 'RFM Customer Segments'] = 'Champions'
  sales_pad.loc[(sales_pad['RFM_Score'] >= 6) & (sales_pad['RFM_Score'] < 9), 'RFM Customer Segments'] = 'Potential Loyalists'
  sales_pad.loc[(sales_pad['RFM_Score'] >= 5) & (sales_pad['RFM_Score'] < 6), 'RFM Customer Segments'] = 'At-Risk Customers'
  sales_pad.loc[(sales_pad['RFM_Score'] >= 4) & (sales_pad['RFM_Score'] < 5), 'RFM Customer Segments'] = 'Cannot Lose'
  sales_pad.loc[(sales_pad['RFM_Score'] >= 3) & (sales_pad['RFM_Score'] < 4), 'RFM Customer Segments'] = 'Lost'

  segment_product_counts = sales_pad.drop_duplicates("card_number").groupby(['Value Segment', 'RFM Customer Segments']).size().reset_index(name = 'Count')
  segment_product_counts = segment_product_counts.sort_values('Count', ascending= False)
  segment_product_counts = segment_product_counts[segment_product_counts['Count'] > 1]

  segment_product_counts["%"] = segment_product_counts["Count"] / n_clientes
  segment_product_counts["sector"] = setor
  segment_product_counts["document_id"] = str(id)

  client_profiles.append(segment_product_counts)

  score_breakdown = sales_pad.describe()[["Recency",'Frequency','MonetaryValue',"RecencyScore",'FrequencyScore','MonetaryScore','RFM_Score']].reset_index()
  score_breakdown["sector"] = setor
  score_breakdown["document_id"] = str(id)

  new_sales.append(sales_pad)
  df_breakdown.append(score_breakdown)

sales    = pd.concat(new_sales)
profiles = pd.concat(client_profiles)
breakdown = pd.concat(df_breakdown)
customers = sales.drop_duplicates(['document_id','card_number'])


In [ ]:
dist_profiles_por_setor = profiles.pivot_table(index='sector',columns='RFM Customer Segments',values='%',aggfunc='mean').reset_index().melt(id_vars = 'sector')
dist_profiles_por_setor.columns = ['sector','RFM Customer Segments','% media setor']
profiles = profiles.merge(dist_profiles_por_setor,on=['sector','RFM Customer Segments'],how='left')
profiles['Δ_ppont_setor'] =  profiles['%'] - profiles['% media setor']

dist_scores_por_setor = customers.groupby("sector")[["RecencyScore","FrequencyScore","MonetaryScore","RFM_Score"]].mean().reset_index()
dist_scores_por_setor.columns = ['sector','Média_setorial_Recency','Média_setorial_Frequency','Média_setorial_Monetary','Média_setorial_RFM']
media_scores = breakdown[breakdown["index"] == 'mean']
media_scores = media_scores.merge(dist_scores_por_setor,on='sector',how='left')

In [ ]:
segment_counts = sales_pad['RFM Customer Segments'].value_counts()

plt.figure(figsize=(10,6))
sns.barplot(x = segment_counts.index, y=segment_counts.values, palette= 'pastel')

champions_color = pastel_colors[9]
colors = [champions_color if segment == 'Champions' else pastel_colors[i] for i, segment in enumerate(segment_counts.index)]
ax = plt.gca()
for i, bar in enumerate(ax.patches):
    bar.set_color(colors[i])

plt.title('Comparison of RFM Segments')
plt.xlabel('RFM Segments')
plt.ylabel('Number of Customers')

ax.yaxis.grid(False)

plt.show()

#### Growth


In [ ]:
sales_core = sales.iloc[:,:8]

sales_core['document_id'] = sales_core['document_id'].astype(str)

In [ ]:
sales_core

In [445]:
monthly_sales_empresa = sales_core.set_index("date_time").pivot_table(index=['sector','document_id'],values='value',columns=pd.Grouper(freq='m'), aggfunc='sum')
monthly_sales_setor = sales_core.set_index("date_time").pivot_table(index='sector',values='value',columns=pd.Grouper(freq='m'), aggfunc='sum')

monthly_growth_empresa = monthly_sales_empresa.T.pct_change().T.reset_index().melt(id_vars=['sector','document_id'])
monthly_growth_empresa.columns = ['sector','document_id','date_time','MoM growth']
monthly_growth_setor = monthly_sales_setor.T.pct_change().T.reset_index().melt(id_vars='sector')
monthly_growth_setor.columns = ['sector','date_time','Sector Growth']


<ipython-input-445-9259519939b9>:4: FutureWarning:

The default fill_method='pad' in DataFrame.pct_change is deprecated and will be removed in a future version. Either fill in any non-leading NA values prior to calling pct_change or specify 'fill_method=None' to not fill NA values.



In [450]:
monthly_sales_empresa.reset_index().melt(id_vars = ["sector","document_id"]).to_excel('monthly_sales_empresa.xlsx')

In [444]:

monthly_sales_empresa = monthly_sales_empresa.T.pct_change().T.reset_index().melt(id_vars=['sector','document_id'])
#monthly_sales_empresa.columns = ['sector','document_id','date_time','MoM growth']
monthly_sales_setor = monthly_sales_setor.T.pct_change().T.reset_index().melt(id_vars='sector')
#monthly_sales_setor.columns = ['sector','date_time','Sector']

<ipython-input-444-7396383d98dd>:1: FutureWarning:

The default fill_method='pad' in DataFrame.pct_change is deprecated and will be removed in a future version. Either fill in any non-leading NA values prior to calling pct_change or specify 'fill_method=None' to not fill NA values.



In [443]:
monthly_sales_empresa

date_time                                                               2022-12-31  \
sector                                             document_id                       
Architectural – Engineering and Surveying Services 3920121792423225234   106615.52   
Bakeries                                           1898616423937443            NaN   
                                                   453832840298988785        40.00   
                                                   5492473503548643259  4015433.52   
                                                   6510304958406240490         NaN   
...                                                                            ...   
Pet Shops, Pet Foods, and Supplies Stores          7347046845893702092    43130.84   
Quick Copy, Reproduction and Blueprinting Services 5727221785363313700    30637.80   
Telecommunications Equipment including telephon... 8028088192899859836    62607.96   
                                                   926930925350410607    100116.00   
Women’s Accessory and Specialty Shops              5615027685943047372    88525.36   

date_time                                                               2023-01-31  \
sector                                             document_id                       
Architectural – Engineering and Surveying Services 3920121792423225234   119699.84   
Bakeries                                           1898616423937443        5578.04   
                                                   453832840298988785        98.00   
                                                   5492473503548643259    20206.40   
                                                   6510304958406240490         NaN   
...                                                                            ...   
Pet Shops, Pet Foods, and Supplies Stores          7347046845893702092    44368.92   
Quick Copy, Reproduction and Blueprinting Services 5727221785363313700    47221.20   
Telecommunications Equipment including telephon... 8028088192899859836    66064.00   
                                                   926930925350410607     91032.40   
Women’s Accessory and Specialty Shops              5615027685943047372   152972.60   

date_time                                                               2023-02-28  \
sector                                             document_id                       
Architectural – Engineering and Surveying Services 3920121792423225234   138331.80   
Bakeries                                           1898616423937443       18175.40   
                                                   453832840298988785     10105.60   
                                                   5492473503548643259    12802.20   
                                                   6510304958406240490         NaN   
...                                                                            ...   
Pet Shops, Pet Foods, and Supplies Stores          7347046845893702092    70022.68   
Quick Copy, Reproduction and Blueprinting Services 5727221785363313700    65759.20   
Telecommunications Equipment including telephon... 8028088192899859836    76262.00   
                                                   926930925350410607     91722.00   
Women’s Accessory and Specialty Shops              5615027685943047372   132654.68   

date_time                                                               2023-03-31  \
sector                                             document_id                       
Architectural – Engineering and Surveying Services 3920121792423225234   387689.52   
Bakeries                                           1898616423937443       18609.72   
                                                   453832840298988785     22993.48   
                                                   5492473503548643259      667.20   
                                                   6510304958406240490         NaN   
...                                      

In [ ]:
monthly_growth_empresa = monthly_growth_empresa.merge(monthly_growth_setor,on=['sector','date_time'],how='left')
monthly_sales_empresa = monthly_sales_empresa.merge(monthly_sales_setor,on=['sector','date_time'],how='left')


In [ ]:
monthly_growth_empresa.to_excel('monthly_growth_empresa.xlsx')
sales.to_excel('sales.xlsx')
profiles.to_excel('profiles.xlsx')
breakdown.to_excel('breakdown.xlsx')
customers.to_excel('customers.xlsx')
media_scores.to_excel('media_scores.xlsx')

In [ ]:
monthly_growth_setor

In [ ]:
sales.growth?

In [ ]:
monthly_sales_empresa

In [ ]:
monthly_sales_empresa.reset_index()

In [ ]:
monthly_growth_empresa

In [ ]:
monthly_growth_setor

In [ ]:
monthly_sales_empresa.T.pct_change().T

In [ ]:
monthly_sales_empresa.pct_change()

In [ ]:
sales_core.set_index("date_time").groupby(['sector','document_id',pd.Grouper(freq='m')]).agg("sum")

In [ ]:
bank

In [ ]:
bank_day = bank.set_index("date_time").groupby(['sector','document_id',pd.Grouper(freq='d'), 'type']).agg("sum").reset_index().iloc[:,:-1]
bank_week = bank.set_index("date_time").groupby(['sector','document_id',pd.Grouper(freq='w'), 'type']).agg("sum").reset_index().iloc[:,:-1]

# daily

outflow_day = bank_day[bank_day["type"] == "pix_out"]
inflow_day = bank_day[bank_day["type"] == "pix_in"]
net_day = bank_day.copy()


outflow_day['cum_out'] = outflow_day.groupby("document_id")['value'].cumsum()
inflow_day['cum_in'] = inflow_day.groupby("document_id")['value'].cumsum()
net_day['cum_net'] = net_day.groupby("document_id")['value'].cumsum()

outflow_day.columns = ['sector', 'document_id', 'date_time', 'type', 'out', 'cum_out']
inflow_day.columns = ['sector', 'document_id', 'date_time', 'type', 'in', 'cum_in']
net_day.columns = ['sector', 'document_id', 'date_time', 'type', 'net', 'cum_net']

# weekly

outflow_week = bank_week[bank_week["type"] == "pix_out"]
inflow_week = bank_week[bank_week["type"] == "pix_in"]
net_week = bank_week.copy()

outflow_week['cum_out'] = outflow_week.groupby("document_id")['value'].cumsum()
inflow_week['cum_in'] = inflow_week.groupby("document_id")['value'].cumsum()
net_week['cum_net'] = net_week.groupby("document_id")['value'].cumsum()

outflow_week.columns = ['sector', 'document_id', 'date_time', 'type', 'out', 'cum_out']
inflow_week.columns = ['sector', 'document_id', 'date_time', 'type', 'in', 'cum_in']
net_week.columns = ['sector', 'document_id', 'date_time', 'type', 'net', 'cum_net']

#### Dist por estado e meio de pagamento

In [ ]:
ad

#### Sample segmentacao

In [ ]:
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
import plotly.io as pio
from datetime import datetime

# Set the default template for Plotly
pio.templates.default = "plotly_white"

# Load the dataset
data = pd.read_csv("rfm_data.csv")
print(data.head())

# Convert 'PurchaseDate' to datetime
data['PurchaseDate'] = pd.to_datetime(data['PurchaseDate'])

# Calculate Recency
data['Recency'] = (datetime.now() - data['PurchaseDate']).dt.days

# Calculate Frequency
frequency_data = data.groupby('CustomerID')['OrderID'].count().reset_index()
frequency_data.rename(columns={'OrderID': 'Frequency'}, inplace=True)
data = data.merge(frequency_data, on='CustomerID', how='left')

# Calculate Monetary Value
monetary_data = data.groupby('CustomerID')['TransactionAmount'].sum().reset_index()
monetary_data.rename(columns={'TransactionAmount': 'MonetaryValue'}, inplace=True)
data = data.merge(monetary_data, on='CustomerID', how='left')

# Define scoring criteria for each RFM value
recency_scores = [5, 4, 3, 2, 1]  # Higher score for lower recency (more recent)
frequency_scores = [1, 2, 3, 4, 5]  # Higher score for higher frequency
monetary_scores = [1, 2, 3, 4, 5]  # Higher score for higher monetary value

# Calculate RFM scores
data['RecencyScore'] = pd.cut(data['Recency'], bins=5, labels=recency_scores)
data['FrequencyScore'] = pd.cut(data['Frequency'], bins=5, labels=frequency_scores)
data['MonetaryScore'] = pd.cut(data['MonetaryValue'], bins=5, labels=monetary_scores)

# Convert RFM scores to numeric type
data['RecencyScore'] = data['RecencyScore'].astype(int)
data['FrequencyScore'] = data['FrequencyScore'].astype(int)
data['MonetaryScore'] = data['MonetaryScore'].astype(int)

# Calculate RFM score by combining the individual scores
data['RFM_Score'] = data['RecencyScore'] + data['FrequencyScore'] + data['MonetaryScore']

# Create RFM segments based on the RFM score
segment_labels = ['Low-Value', 'Mid-Value', 'High-Value']
data['Value Segment'] = pd.qcut(data['RFM_Score'], q=3, labels=segment_labels)

# RFM Segment Distribution
segment_counts = data['Value Segment'].value_counts().reset_index()
segment_counts.columns = ['Value Segment', 'Count']

# Define the pastel color palette
pastel_colors = px.colors.qualitative.Pastel

# Create the bar chart with pastel colors
fig_segment_dist = px.bar(segment_counts, x='Value Segment', y='Count',
                          color='Value Segment', color_discrete_sequence=pastel_colors,
                          title='RFM Value Segment Distribution')

# Update the layout
fig_segment_dist.update_layout(xaxis_title='RFM Value Segment',
                               yaxis_title='Count',
                               showlegend=False)

# Show the figure
fig_segment_dist.show()

# Create a new column for RFM Customer Segments
data['RFM Customer Segments'] = ''

# Assign RFM segments based on the RFM score
data.loc[data['RFM_Score'] >= 9, 'RFM Customer Segments'] = 'Champions'
data.loc[(data['RFM_Score'] >= 6) & (data['RFM_Score'] < 9), 'RFM Customer Segments'] = 'Potential Loyalists'
data.loc[(data['RFM_Score'] >= 5) & (data['RFM_Score'] < 6), 'RFM Customer Segments'] = 'At Risk Customers'
data.loc[(data['RFM_Score'] >= 4) & (data['RFM_Score'] < 5), 'RFM Customer Segments'] = "Can't Lose"
data.loc[(data['RFM_Score'] >= 3) & (data['RFM_Score'] < 4), 'RFM Customer Segments'] = "Lost"

# Count of customers in each segment and value segment
segment_product_counts = data.groupby(['Value Segment', 'RFM Customer Segments']).size().reset_index(name='Count')
segment_product_counts = segment_product_counts.sort_values('Count', ascending=False)

# Create a treemap for RFM Customer Segments by Value
fig_treemap_segment_product = px.treemap(segment_product_counts,
                                         path=['Value Segment', 'RFM Customer Segments'],
                                         values='Count',
                                         color='Value Segment', color_discrete_sequence=pastel_colors,
                                         title='RFM Customer Segments by Value')
fig_treemap_segment_product.show()

# Filter the data to include only the customers in the Champions segment
champions_segment = data[data['RFM Customer Segments'] == 'Champions']

# Create a box plot for RFM values within the Champions segment
champions_segment_fig = go.Figure()
champions_segment_fig.add_trace(go.Box(y=champions_segment['RecencyScore'], name='Recency'))
champions_segment_fig.add_trace(go.Box(y=champions_segment['FrequencyScore'], name='Frequency'))
champions_segment_fig.add_trace(go.Box(y=champions_segment['MonetaryScore'], name='Monetary'))

champions_segment_fig.update_layout(title='Distribution of RFM Values within Champions Segment',
                                    yaxis_title='RFM Value',
                                    showlegend=True)

champions_segment_fig.show()

# Calculate the correlation matrix for RFM values within the Champions segment
correlation_matrix = champions_segment[['RecencyScore', 'FrequencyScore', 'MonetaryScore']].corr()

# Visualize the correlation matrix using a heatmap
fig_corr_heatmap = go.Figure(data=go.Heatmap(
                   z=correlation_matrix.values,
                   x=correlation_matrix.columns,
                   y=correlation_matrix.columns,
                   colorscale='RdBu',
                   colorbar=dict(title='Correlation')))

fig_corr_heatmap.update_layout(title='Correlation Matrix of RFM Values within Champions Segment')

fig_corr_heatmap.show()

# Define the pastel color palette
pastel_colors = px.colors.qualitative.Pastel

# Segment counts for RFM Customer Segments
segment_counts = data['RFM Customer Segments'].value_counts()

# Create a bar chart to compare segment counts
comparison_fig = go.Figure(data=[go.Bar(x=segment_counts.index, y=segment_counts.values,
                                        marker=dict(color=pastel_colors))])

# Set the color of the Champions segment as a different color
champions_color = 'rgb(158, 202, 225)'
comparison_fig.update_traces(marker_color=[champions_color if segment == 'Champions' else pastel_colors[i]
                                           for i, segment in enumerate(segment_counts.index)],
                             marker_line_color='rgb(8, 48, 107)',
                             marker_line_width=1.5, opacity=0.6)

# Update the layout
comparison_fig.update_layout(title='Comparison of RFM Segments',
                             xaxis_title='RFM Segments',
                             yaxis_title='Number of Customers',
                             showlegend=False)

# Show the figure
comparison_fig.show()

# Calculate the average Recency, Frequency, and Monetary scores for each segment
segment_scores = data.groupby('RFM Customer Segments')[['RecencyScore', 'FrequencyScore', 'MonetaryScore']].mean().reset_index()

# Create a grouped bar chart to compare segment scores
fig = go.Figure()

# Add bars for Recency score
fig.add_trace(go.Bar(
    x=segment_scores['RFM Customer Segments'],
    y=segment_scores['RecencyScore'],
    name='Recency Score',
    marker_color='rgb(158,202,225)'
))

# Add bars for Frequency score
fig.add_trace(go.Bar(
    x=segment_scores['RFM Customer Segments'],
    y=segment_scores['FrequencyScore'],
    name='Frequency Score',
    marker_color='rgb(94,158,217)'
))

# Add bars for Monetary score
fig.add_trace(go.Bar(
    x=segment_scores['RFM Customer Segments'],
    y=segment_scores['MonetaryScore'],
    name='Monetary Score',
    marker_color='rgb(32,102,148)'
))

# Update the layout
fig.update_layout(
    title='Comparison of RFM Segments based on Recency, Frequency, and Monetary Scores',
    xaxis_title='RFM Segments',
    yaxis_title='Score',
    barmode='group',
    showlegend=True
)

fig.show()